In [1]:
import numpy as np
print(np.__version__)  # 버전이 2.0 이하인지 확인


2.0.0


In [13]:
import random
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from scipy import ndimage
import sklearn
from sklearn.model_selection import train_test_split
import sktime
from sktime.datatypes._panel._convert import from_3d_numpy_to_nested
from sklearn.metrics import accuracy_score, classification_report

from sktime.datatypes._panel._convert import from_2d_array_to_nested

from sktime.transformations.panel.rocket import MiniRocket
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

from numba import config



In [3]:
def maximum(raw, box_size, mode='nearest'): # Maximum Filter
    raw_maximum=ndimage.maximum_filter(raw,box_size,mode=mode)
    return raw_maximum.real

def minimum(raw, box_size, mode='nearest'): # Maximum Filter
    raw_minimum=ndimage.minimum_filter(raw,box_size,mode=mode)
    return raw_minimum.real

def denoise_fft(data, ifftn): # Fast Fourier Transformation
    fft_signal = np.fft.fft(data)
    
    # Reconstruct the original signal
    fft_signal[ifftn:len(fft_signal)//2]=0
    fft_signal[len(fft_signal)//2:-ifftn]=0
    reconstructed_signal = np.fft.ifft(fft_signal)
    
    return reconstructed_signal.real

def smooth(x,beta): # Kaiser Window Smoothing
    window_len=11  # extending the data at beginning and at the end to apply the window at the borders

    s = np.r_[x[window_len-1:0:-1],x,x[-1:-window_len:-1]]
    w = np.kaiser(window_len,beta)
    y = np.convolve(w/w.sum(),s,mode='valid')
    return y[5:len(y)-5]

In [16]:
import joblib

# 모델과 인코딩 정보를 불러오는 코드
clf = joblib.load('xgb_clf.joblib')  # 저장된 XGBoost 모델 불러오기
encd = joblib.load('mnr_encd.joblib')   # 저장된 인코딩 정보 불러오기

In [ ]:
# 모델을 불러온 후 예측을 수행할 수 있습니다.
# 예를 들어, 새로운 데이터에 대해 예측:
y_pred = clf.predict(X_new)

In [27]:
num_list = []
time_series = []

df = pd.read_csv('data4/rest/rest2.csv', header=None, names=['time', 'a0', 'a1', 'a2', 'a3', 'a4', 'a5'])

df['time'] = pd.to_datetime(df['time'])

sec = 15
while(True):
    if sec > 50:
        break
    
    start_time = df['time'].iloc[0] + pd.Timedelta(seconds=sec)
    end_time = df['time'].iloc[0] + pd.Timedelta(seconds=sec+0.4)
    df_filtered = df[(df['time'] >= start_time) & (df['time'] <= end_time)]

    depths_list = []
    for i in range(6):
        column = f'a{i}'  # 필터링된 데이터의 각 채널 컬럼명
        depths = smooth(denoise_fft(maximum(df_filtered[column], 5)-minimum(df_filtered[column], 5), 5), 5)
        diff = np.diff(depths)
        depths_list.append(depths)
        depths_list.append(diff)
    df_2d = pd.DataFrame(np.hstack(depths_list))

    df_2d_cleaned = df_2d.dropna(axis=1, how='any')

    nested = from_2d_array_to_nested(df_2d_cleaned)

    # nested = encd.transform(nested)

    # result = df_2d_cleaned.to_numpy().flatten()

    # print(clf.predict(result))

    # num_list.append([model.predict(result)[0], sec, sec+1, np.round(model.predict_proba(result), 3)])

    sec+=0.1

In [46]:
df_2d_cleaned

,0
0,0.977746
1,0.977461
2,0.976700
3,0.975704
4,0.974770
...,...
2245,-0.016837
2246,-0.016283
2247,-0.014426
2248,-0.010946


In [56]:
df_2d_cleaned
repeated_df = pd.concat([df_2d_cleaned[0]] * 336, axis=1)

# 2. 새로운 열 이름 설정 (선택 사항)
repeated_df.columns = [f'{i-1+1}' for i in range(336)]
print(repeated_df)
# nested = from_2d_array_to_nested(repeated_df)

             0         1         2         3         4         5         6  \
0     0.977746  0.977746  0.977746  0.977746  0.977746  0.977746  0.977746   
1     0.977461  0.977461  0.977461  0.977461  0.977461  0.977461  0.977461   
2     0.976700  0.976700  0.976700  0.976700  0.976700  0.976700  0.976700   
3     0.975704  0.975704  0.975704  0.975704  0.975704  0.975704  0.975704   
4     0.974770  0.974770  0.974770  0.974770  0.974770  0.974770  0.974770   
...        ...       ...       ...       ...       ...       ...       ...   
2245 -0.016837 -0.016837 -0.016837 -0.016837 -0.016837 -0.016837 -0.016837   
2246 -0.016283 -0.016283 -0.016283 -0.016283 -0.016283 -0.016283 -0.016283   
2247 -0.014426 -0.014426 -0.014426 -0.014426 -0.014426 -0.014426 -0.014426   
2248 -0.010946 -0.010946 -0.010946 -0.010946 -0.010946 -0.010946 -0.010946   
2249 -0.005939 -0.005939 -0.005939 -0.005939 -0.005939 -0.005939 -0.005939   

             7         8         9  ...       326       327    

In [52]:
nested

,0
0,0 0.977746 1 0.977746 2 0.97774...
1,0 0.977461 1 0.977461 2 0.97746...
2,0 0.9767 1 0.9767 2 0.9767 3 ...
3,0 0.975704 1 0.975704 2 0.97570...
4,0 0.97477 1 0.97477 2 0.97477 3...
...,...
2245,0 -0.016837 1 -0.016837 2 -0.01683...
2246,0 -0.016283 1 -0.016283 2 -0.01628...
2247,0 -0.014426 1 -0.014426 2 -0.01442...
2248,0 -0.010946 1 -0.010946 2 -0.01094...


In [51]:
nested.shape

(2250, 1)

In [41]:
nested_expanded = np.tile(nested, (1, 336))
nested_expanded = nested_expanded[:, :, np.newaxis]
print(nested_expanded.shape)

(2250, 336, 1)


In [42]:
nested
nested_expanded = np.tile(nested, (1, 336))
nested_expanded = nested_expanded[:, :, np.newaxis]
nested1 = encd.transform(nested_expanded)

ValueError: Indices for islice() must be None or an integer: 0 <= x <= sys.maxsize.